[View in Colaboratory](https://colab.research.google.com/github/sl2820/Deep_Learning_Study/blob/master/RNN_Practice.ipynb)

# Sequence Classification


In [11]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow.contrib.rnn as rnn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print ("Packages Loaded")

Packages Loaded


### Get MNIST


In [12]:
mnist = input_data.read_data_sets("data/", one_hot = True)
trainimgs, trainlabels, testimgs, testlabels = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
ntrain, ntest, dim, nclasses \
= trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1], trainlabels.shape[1]
print("MNIST LOADED")
print ("TF Version is: %s" % (tf.__version__))

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST LOADED
TF Version is: 1.11.0


### Define Model
LSTM 자체는 텐서플로우 것 사용, Input 과 Output에 대한 것만 Define 하면 됌.

In [0]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
nsteps = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput, dimhidden])), #28 --> 128
    'out': tf.Variable(tf.random_normal([dimhidden, dimoutput]))  # 128 --> 10
}

biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}

### Define Function

In [16]:
def _RNN(_X, _istate,_W,_b,_nsteps,_name):
  # 1. Permute input from [batchsize, nsteps, diminput] --> [nsteps, batchsize, diminput]
  # Current _X = 784 dim
  _X = tf.transpose(_X, [1,0,2])   
  
  # 2. Reshape input to [nsteps*batchsize, diminput]
  _X = tf.reshape(_X, [-1, diminput])
  
  # 3. Input to hidden layer
  _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
  
  # 4. Split data to 'nsteps' chunks --> LIST
  _Hsplit = tf.split(_H, _nsteps, axis=0)
  # 5. Get LSTM's final output (_LSTM_O) and State (_LSTM_S)
  with tf.variable_scope(_name):
    # RNN <-- tf.contrib.rnn
    lstm_cell = rnn.BasicLSTMCell(dimhidden, forget_bias=1.0, state_is_tuple=False, reuse=True)
    _LSTM_O, _LSTM_S = rnn.static_rnn(lstm_cell, _Hsplit, initial_state=_istate)
    
  _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']  #_LSTM_O[-1] --> 맨 마지막 아웃풋
  return{
      'X': _X, 'H': _H, 'Hsplit': _Hsplit, 'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O
  }

print("Function Ready")

Function Ready


### Define Graph

In [17]:
learning_rate = 0.001
x = tf.placeholder("float", [None, nsteps, diminput])
istate = tf.placeholder("float", [None, 2*dimhidden])
y = tf.placeholder("float", [None, dimoutput])
myrnn = _RNN(x, istate, weights, biases, nsteps, 'basic')
pred = myrnn['O']
celoss = tf.nn.softmax_cross_entropy_with_logits
cost = tf.reduce_mean(celoss(logits=pred,labels = y))
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost)
accr = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init = tf.global_variables_initializer()
print ("Network Ready")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Network Ready


### Run

In [0]:
t_epochs = 5
batch_size = 128
display_step = 1
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(init)

### Optimization

In [22]:
#Optimization Starts
for epoch in range(t_epochs):
  avg_cost = 0.
  total_batch = int(mnist.train.num_examples/batch_size)
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape((batch_size,nsteps,diminput))
    feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
    sess.run(optm, feed_dict=feeds)
    avg_cost += sess.run(cost, feed_dict= feeds)/total_batch
    
  if epoch % display_step ==0:
    print ("Epoch: %03d/%03d Cost: %.9f" % (epoch, t_epochs, avg_cost))
    feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
    train_acc = sess.run(accr, feed_dict=feeds)
    print("Train Accuracy: %.3f" %(train_acc))
    testimgs = testimgs.reshape((ntest,nsteps,diminput))
    feeds = {x: testimgs, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
    test_acc = sess.run(accr,feed_dict = feeds)
    print("Test Accuracy: %.3f" % (test_acc))

print("Optimization Done")

Epoch: 000/005 Cost: 0.154996177
Train Accuracy: 0.953
Test Accuracy: 0.955
Epoch: 001/005 Cost: 0.100688199
Train Accuracy: 0.984
Test Accuracy: 0.962
Epoch: 002/005 Cost: 0.075377963
Train Accuracy: 0.969
Test Accuracy: 0.961
Epoch: 003/005 Cost: 0.059422904
Train Accuracy: 0.992
Test Accuracy: 0.973
Epoch: 004/005 Cost: 0.047436275
Train Accuracy: 0.984
Test Accuracy: 0.979
Optimization Done


여기까지가 RNN 8:27까지 한것 

### What if we use smaller number of Sequences?
내일 해보기